# 0. ABSTRACT:

## In this ETL prject, we wanted to obtain a data set would help us answer the question "Do housing prices and other factors fluctuate due to the quality of the schools in the area?".  In this scenario, our hypothesis would be that our dependent, housing prices, do have a notable correlation with the quality of the schools in the are.

## To obtain our data set, we have scrapped realtor.com and usnews.com, respectively, to obtain information on house prices/ charaacteristis in comparrison to school qualities in different areas around St. Louis.  We will Explain or Extract, Transform, and Load procedures in further detail below.


# 1. EXTRACT:

## First we have to extract som einfo for shollls in the area.  To do this, we located a website called US News to get a comprehensive list of the schools and districts in the area .  The preocess of this data gathering is explained in further detail below.

In [1]:
# Import dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Set up the executable path and browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Create base URLs
url = 'https://www.usnews.com/education/best-high-schools/missouri/rankings'
small_url = "https://www.usnews.com/"
browser.visit(url)

In [4]:
# Set Counter and done variable for the while loop
done = False
counter = 0

# Execute while loop to fully load the downward extending page with Splinter
while done == False:

    try:
        browser.click_link_by_partial_text('Load More')
        counter += 1
          
    except:
        print(f"Page Load Complete...  {counter} page clicks!!!")
        done = True

Page Load Complete...  29 page clicks!!!


In [5]:
# Envoke beautiful soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


# Set base level list for the tag navigation
school_district_fields = soup.find_all('div', class_='search-result-content')



# Set Empty lists
school_name_list = []
city_district_list = []
MO_ranking_list = []
national_ranking_list = []
page_links_list = []


# Loop through all of the bases in school_district_data to get teh needed info
for field in school_district_fields:
    
    
    try:
          
            # Get MO rankings
            span = field.find('span', class_='text-normal')
            MO_ranking = span.text.strip()
        
            # Make sure we avoud the unranced schoold with this conditional
            if len(MO_ranking) > 10:
                
                
                    MO_ranking_list.append(MO_ranking)


                    # Get names
                    h3 = field.find('h3')
                    school_name = h3.text.strip()
                    school_name_list.append(school_name)


                    # Get city and district
                    p = field.find('p', class_='text-small')
                    city_district = p.text.strip()
                    city_district_list.append(city_district)



                    # Get national rankings
                    span = field.find('span', class_='text-small')
                    national_ranking = span.text.strip()
                    national_ranking_list.append(national_ranking)


                    # Get the link to access the full address and zip code
                    h3_link = field.find('h3', class_="block-tight")
                    a = h3_link.find('a')
                    school_page_link = a['href']
                    full_page_link = (small_url + school_page_link)
                    page_links_list.append(full_page_link)
                    
                    
                    
                    
                    print(f'Now logging: {school_name}...\nFull link:  {full_page_link}')
                          
                    print(f'Log {len(page_links_list)} complete.\n')

    
    except AttributeError:
        
        print('No text/ bad link...  Skipping')
        
        
        

Now logging: Lafayette Sr. High...
Full link:  https://www.usnews.com//education/best-high-schools/missouri/districts/rockwood-r-vi/lafayette-sr-high-11883
Log 1 complete.

Now logging: Smithville High...
Full link:  https://www.usnews.com//education/best-high-schools/missouri/districts/smithville-r-ii/smithville-high-11910
Log 2 complete.

Now logging: Rockwood Summit Sr. High...
Full link:  https://www.usnews.com//education/best-high-schools/missouri/districts/rockwood-r-vi/rockwood-summit-sr-high-11881
Log 3 complete.

Now logging: Eureka Sr. High...
Full link:  https://www.usnews.com//education/best-high-schools/missouri/districts/rockwood-r-vi/eureka-sr-high-11882
Log 4 complete.

Now logging: Metro Academic and Classical High School...
Full link:  https://www.usnews.com//education/best-high-schools/missouri/districts/st-louis-public-schools/metro-academic-and-classical-high-school-11941
Log 5 complete.

Now logging: West High...
Full link:  https://www.usnews.com//education/best-

# 2. TRANSFORM

## Great, we recived almost 300 auality results for the schools data.  Next we will clean and reshape our data to formulate it in a manner that can help tell a comprehensive story of the future findings.

In [6]:
# Set empty list and counter
address_list = []
counter = 0

# Loop through and load the pages to retrieve the full address and zip codes
for link in page_links_list:
    

    browser.visit(link)

    # Get the full address and zip code
    zip_html = browser.html
    zip_soup = BeautifulSoup(zip_html, 'html.parser')
    zip_p = zip_soup.find('p', class_="block-normal")
    address = zip_p.text.strip()
    
    address_list.append(address)  
    counter += 1
    
    
    
    print(address + '        ' + str(counter))

17050 Clayton Rd, Wildwood, Missouri 63011
 | (636) 733-4100        1
645 S Commercial Ave, Smithville, Missouri 64089
 | (816) 532-0405        2
1780 Hawkins Rd, Fenton, Missouri 63026
 | (636) 891-6800        3
4525 Hwy 109, Eureka, Missouri 63025
 | (636) 733-3100        4
4015 Mcpherson Ave, St Louis, Missouri 63108
 | (314) 534-3894        5
14653 Clayton Rd, Ballwin, Missouri 63011
 | (314) 415-7500        6
5000 S. Lindbergh, St. Louis, Missouri 63126
 | (314) 729-2410        7
2351 Clarkson Rd, Chesterfield, Missouri 63017
 | (636) 891-6000        8
8309 Mackenzie Rd, St Louis, Missouri 63123
 | (314) 638-6330        9
1251 Turtle Creek Dr, O'fallon, Missouri 63366
 | (636) 379-0300        10
2275 Sommers Road, Lake St. Louis, Missouri 63367
 | (636) 561-0075        11
1547 S. Theresa Av, St. Louis, Missouri 63104
 | (314) 696-2290        12
369 N Woods Mill Rd, Chesterfield, Missouri 63017
 | (314) 415-7900        13
504 S Washington, Knob Noster, Missouri 65336
 | (660) 563-2

28176 Hwy Ww, Clarksville, Missouri 63336
 | (573) 242-3546        115
1022 S Chestnut, Cameron, Missouri 64429
 | (816) 882-1036        116
245 State Rd N, Macks Creek, Missouri 65786
 | (573) 363-5911        117
1073 E Vine St, Sullivan, Missouri 63080
 | (573) 468-5171        118
1 Main St, Valley Park, Missouri 63088
 | (636) 923-3613        119
805 E Hwy 72, Fredericktown, Missouri 63645
 | (573) 783-3628        120
3710 S Jefferson, Springfield, Missouri 65807
 | (417) 523-8500        121
14520 Crump Ln, Ashland, Missouri 65010
 | (573) 657-2144        122
Rte 1 Box 1187, Myrtle, Missouri 65778
 | (417) 938-4212        123
100 Mustang Dr, Anderson, Missouri 64831
 | (417) 845-3322        124
107 School St, Pilot Grove, Missouri 65276
 | (660) 834-4415        125
176 Bearcat Crossing, Hermann, Missouri 65041
 | (573) 486-5425        126
55701 State Hwy 6, Edina, Missouri 63537
 | (660) 397-2231        127
101 S Pine St, Eldon, Missouri 65026
 | (573) 392-8010        128
1 Trojan D

1901 S Main, Holden, Missouri 64040
 | (816) 732-5523        231
423 W Pine, Houston, Missouri 65483
 | (417) 967-3024        232
300 N Grand, King City, Missouri 64463
 | (660) 535-4319        233
101 E Adriatic, Kingsville, Missouri 64061
 | (816) 597-3422        234
807a W 31st, Higginsville, Missouri 64037
 | (660) 584-3661        235
125 College Avenue, Licking, Missouri 65542
 | (573) 674-2711        236
15533 Hwy Kk, Purdin, Missouri 64674
 | (660) 244-5035        237
141 Wildcat Drive, Linn, Missouri 65051
 | (573) 897-4216        238
4700 S State Highway 125, Rogersville, Missouri 65742
 | (417) 753-2813        239
3321 W Georgia St, Louisiana, Missouri 63353
 | (573) 754-6181        240
309 Thomas St, Madison, Missouri 65263
 | (660) 291-4515        241
5309 S Main, Morrisville, Missouri 65710
 | (417) 376-2216        242
805 S Miami, Marshall, Missouri 65340
 | (660) 886-2244        243
601 W Main, Maysville, Missouri 64469
 | (816) 449-2154        244
101 W Crandall, Meadvi

In [7]:
# Split the list by the pipe to make two dirivative addresss and phone number lists
full_address_list_dirty, phone_number_list = zip(*([address.split(' | ') for address in address_list]))


# Check list
full_address_list_dirty

('17050 Clayton Rd, Wildwood, Missouri 63011\n',
 '645 S Commercial Ave, Smithville, Missouri 64089\n',
 '1780 Hawkins Rd, Fenton, Missouri 63026\n',
 '4525 Hwy 109, Eureka, Missouri 63025\n',
 '4015 Mcpherson Ave, St Louis, Missouri 63108\n',
 '14653 Clayton Rd, Ballwin, Missouri 63011\n',
 '5000 S. Lindbergh, St. Louis, Missouri 63126\n',
 '2351 Clarkson Rd, Chesterfield, Missouri 63017\n',
 '8309 Mackenzie Rd, St Louis, Missouri 63123\n',
 "1251 Turtle Creek Dr, O'fallon, Missouri 63366\n",
 '2275 Sommers Road, Lake St. Louis, Missouri 63367\n',
 '1547 S. Theresa Av, St. Louis, Missouri 63104\n',
 '369 N Woods Mill Rd, Chesterfield, Missouri 63017\n',
 '504 S Washington, Knob Noster, Missouri 65336\n',
 '#1 Mark Twain Circle, Clayton, Missouri 63105\n',
 '7539 Manchester Rd, Maplewood, Missouri 63143\n',
 '801 Hanna Rd, Manchester, Missouri 63021\n',
 '3601 Droste Rd, St Charles, Missouri 63301\n',
 '2111 Woodland, Kansas City, Missouri 64108\n',
 '100 Selma Ave, St Louis, Missouri 

In [8]:
# Finish cleaning for the full address
full_address_list_clean = [address.strip('\n') for address in full_address_list_dirty]


# Check list
full_address_list_clean

['17050 Clayton Rd, Wildwood, Missouri 63011',
 '645 S Commercial Ave, Smithville, Missouri 64089',
 '1780 Hawkins Rd, Fenton, Missouri 63026',
 '4525 Hwy 109, Eureka, Missouri 63025',
 '4015 Mcpherson Ave, St Louis, Missouri 63108',
 '14653 Clayton Rd, Ballwin, Missouri 63011',
 '5000 S. Lindbergh, St. Louis, Missouri 63126',
 '2351 Clarkson Rd, Chesterfield, Missouri 63017',
 '8309 Mackenzie Rd, St Louis, Missouri 63123',
 "1251 Turtle Creek Dr, O'fallon, Missouri 63366",
 '2275 Sommers Road, Lake St. Louis, Missouri 63367',
 '1547 S. Theresa Av, St. Louis, Missouri 63104',
 '369 N Woods Mill Rd, Chesterfield, Missouri 63017',
 '504 S Washington, Knob Noster, Missouri 65336',
 '#1 Mark Twain Circle, Clayton, Missouri 63105',
 '7539 Manchester Rd, Maplewood, Missouri 63143',
 '801 Hanna Rd, Manchester, Missouri 63021',
 '3601 Droste Rd, St Charles, Missouri 63301',
 '2111 Woodland, Kansas City, Missouri 64108',
 '100 Selma Ave, St Louis, Missouri 63119',
 '600 1st Executive Ave, St Pe

In [9]:
# Check list
phone_number_list

('(636) 733-4100',
 '(816) 532-0405',
 '(636) 891-6800',
 '(636) 733-3100',
 '(314) 534-3894',
 '(314) 415-7500',
 '(314) 729-2410',
 '(636) 891-6000',
 '(314) 638-6330',
 '(636) 379-0300',
 '(636) 561-0075',
 '(314) 696-2290',
 '(314) 415-7900',
 '(660) 563-2283',
 '(314) 854-6600',
 '(314) 644-4401',
 '(314) 415-7700',
 '(636) 443-4200',
 '(816) 418-3000',
 '(314) 963-6400',
 '(636) 477-2400',
 '(816) 521-5350',
 '(636) 978-1212',
 '(816) 874-3400',
 '(636) 272-4447',
 '(816) 650-7063',
 '(636) 851-4700',
 '(816) 874-3500',
 '(816) 630-9210',
 '(660) 851-5300',
 '(417) 523-9600',
 '(573) 214-3100',
 '(314) 467-7000',
 '(573) 333-6110',
 '(314) 213-6110',
 '(314) 213-8051',
 '(636) 327-3988',
 '(816) 847-5000',
 '(816) 359-4110',
 '(816) 986-3000',
 '(816) 986-2000',
 '(816) 736-5340',
 '(660) 665-4631',
 '(816) 736-5500',
 '(314) 415-7600',
 '(816) 321-5310',
 '(816) 671-4080',
 '(816) 628-4585',
 '(636) 327-3876',
 '(816) 858-2822',
 '(417) 582-5901',
 '(636) 274-0555',
 '(816) 321-

In [10]:
# Create zipcode list
zipcode_list = [x[(len(x) - 5):] for x in full_address_list_clean]


# Check list
zipcode_list

['63011',
 '64089',
 '63026',
 '63025',
 '63108',
 '63011',
 '63126',
 '63017',
 '63123',
 '63366',
 '63367',
 '63104',
 '63017',
 '65336',
 '63105',
 '63143',
 '63021',
 '63301',
 '64108',
 '63119',
 '63376',
 '64055',
 '63376',
 '64015',
 '63366',
 '64058',
 '63304',
 '64014',
 '64024',
 '65301',
 '65802',
 '65203',
 '63129',
 '63830',
 '63122',
 '63043',
 '63385',
 '64029',
 '64153',
 '64086',
 '64063',
 '64068',
 '63501',
 '64068',
 '63146',
 '64116',
 '64501',
 '64060',
 '63385',
 '64079',
 '65721',
 '63016',
 '64118',
 '63303',
 '65203',
 '63703',
 '63139',
 '65583',
 '64098',
 '64150',
 '65652',
 '63555',
 '64119',
 '64050',
 '64053',
 '64120',
 '65605',
 '64131',
 '63301',
 '65233',
 '65453',
 '64468',
 '64012',
 '64850',
 '63755',
 '63090',
 '64078',
 '63461',
 '65065',
 '63125',
 '63901',
 '65616',
 '65613',
 '63125',
 '63010',
 '64131',
 '65791',
 '64505',
 '64485',
 '63383',
 '64739',
 '65738',
 '65708',
 '63877',
 '63123',
 '63556',
 '63052',
 '63104',
 '63845',
 '64075',


In [11]:
# Clean the national ranking list
national_ranking_strip_list = [ranking.strip(' in National Rankings') for ranking in national_ranking_list]
clean_national_ranking_final = [ranking.strip('#') for ranking in national_ranking_strip_list]


# Check list
clean_national_ranking_final

['431',
 '531',
 '621',
 '681',
 '799',
 '823',
 '940',
 '947',
 '977',
 '1,196',
 '1,205',
 '1,241',
 '1,384',
 '1,649',
 '1,826',
 '1,837',
 '1,867',
 '1,906',
 '1,923',
 '1,927',
 '1,987',
 '2,018',
 '2,115',
 '2,155',
 '2,157',
 '2,216',
 '2,248',
 '2,275',
 '2,438',
 '2,481',
 '2,495',
 '2,532',
 '2,621',
 '2,680',
 '2,736',
 '2,783',
 '3,140',
 '3,173',
 '3,177',
 '3,207',
 '3,315',
 '3,332',
 '3,359',
 '3,465',
 '3,471',
 '3,572',
 '3,645',
 '3,802',
 '3,848',
 '3,879',
 '3,951',
 '3,954',
 '4,177',
 '4,251',
 '4,360',
 '4,410',
 '4,591',
 '4,709',
 '4,735',
 '4,857',
 '4,875',
 '4,922',
 '5,025',
 '5,084',
 '5,234',
 '5,250',
 '5,286',
 '5,368',
 '5,427',
 '5,471',
 '5,476',
 '5,512',
 '5,581',
 '5,674',
 '5,797',
 '6,114',
 '6,200',
 '6,211',
 '6,213',
 '6,257',
 '6,292',
 '6,378',
 '6,430',
 '6,533',
 '6,591',
 '6,795',
 '7,092',
 '7,206',
 '7,398',
 '7,414',
 '7,470',
 '7,485',
 '7,538',
 '7,630',
 '7,642',
 '7,655',
 '7,718',
 '7,820',
 '7,844',
 '7,900',
 '7,907',
 '7,911'

In [12]:
# Clean the MO ranking list
MO_ranking_strip_list = [ranking.strip(' in Missouri Rankings') for ranking in MO_ranking_list]
clean_MO_ranking_final = [ranking.strip('#') for ranking in MO_ranking_strip_list]


# Check list
clean_MO_ranking_final

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '13

In [35]:
# Create te city and district lists
city_list, district_list = zip(*([cit_dis.split(' | ') for cit_dis in city_district_list]))


# Check list
city_list

('Wildwood, MO',
 'Smithville, MO',
 'Fenton, MO',
 'Eureka, MO',
 'St Louis, MO',
 'Ballwin, MO',
 'St. Louis, MO',
 'Chesterfield, MO',
 'St Louis, MO',
 "O'fallon, MO",
 'Lake St. Louis, MO',
 'St. Louis, MO',
 'Chesterfield, MO',
 'Knob Noster, MO',
 'Clayton, MO',
 'Maplewood, MO',
 'Manchester, MO',
 'St Charles, MO',
 'Kansas City, MO',
 'St Louis, MO',
 'St Peters, MO',
 'Independence, MO',
 'St Peters, MO',
 'Blue Springs, MO',
 "O'fallon, MO",
 'Independence, MO',
 'St Charles, MO',
 'Blue Springs, MO',
 'Excelsior Springs, MO',
 'Sedalia, MO',
 'Springfield, MO',
 'Columbia, MO',
 'St Louis, MO',
 'Caruthersville, MO',
 'Kirkwood, MO',
 'Maryland Heights, MO',
 'Wentzville, MO',
 'Grain Valley, MO',
 'Kansas City, MO',
 "Lee's Summit, MO",
 "Lee's Summit, MO",
 'Liberty, MO',
 'Kirksville, MO',
 'Liberty, MO',
 'St Louis, MO',
 'Kansas City, MO',
 'St Joseph, MO',
 'Kearney, MO',
 'Wentzville, MO',
 'Platte City, MO',
 'Ozark, MO',
 'Cedar Hill, MO',
 'Kansas City, MO',
 'St

In [36]:
city_list_clean = [city[:(len(city)-4)] for city in city_list]

city_list_clean

['Wildwood',
 'Smithville',
 'Fenton',
 'Eureka',
 'St Louis',
 'Ballwin',
 'St. Louis',
 'Chesterfield',
 'St Louis',
 "O'fallon",
 'Lake St. Louis',
 'St. Louis',
 'Chesterfield',
 'Knob Noster',
 'Clayton',
 'Maplewood',
 'Manchester',
 'St Charles',
 'Kansas City',
 'St Louis',
 'St Peters',
 'Independence',
 'St Peters',
 'Blue Springs',
 "O'fallon",
 'Independence',
 'St Charles',
 'Blue Springs',
 'Excelsior Springs',
 'Sedalia',
 'Springfield',
 'Columbia',
 'St Louis',
 'Caruthersville',
 'Kirkwood',
 'Maryland Heights',
 'Wentzville',
 'Grain Valley',
 'Kansas City',
 "Lee's Summit",
 "Lee's Summit",
 'Liberty',
 'Kirksville',
 'Liberty',
 'St Louis',
 'Kansas City',
 'St Joseph',
 'Kearney',
 'Wentzville',
 'Platte City',
 'Ozark',
 'Cedar Hill',
 'Kansas City',
 'St Charles',
 'Columbia',
 'Cape Girardeau',
 'St. Louis',
 'Waynesville',
 'Weston',
 'Riverside',
 'Fordland',
 'Memphis',
 'Kansas City',
 'Independence',
 'Independnce',
 'Kansas City',
 'Aurora',
 'Kansas City

In [37]:
state_list = [state[(len(state)-2):] for state in city_list]

state_list

['MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',
 'MO',

In [38]:
# Check list
district_list

('Rockwood R-Vi',
 'Smithville R-II',
 'Rockwood R-Vi',
 'Rockwood R-Vi',
 'St. Louis Public Schools',
 'Parkway C-2',
 'Lindbergh Schools',
 'Rockwood R-Vi',
 'Affton 101',
 'Ft. Zumwalt R-II',
 'Wentzville R-Iv',
 'St. Louis Public Schools',
 'Parkway C-2',
 'Knob Noster R-Viii',
 'Clayton',
 'Maplewood-Richmond Heights',
 'Parkway C-2',
 'St. Charles R-Vi',
 'Kansas City 33',
 'Webster Groves',
 'Ft. Zumwalt R-II',
 'Independence 30',
 'Ft. Zumwalt R-II',
 'Blue Springs R-Iv',
 'Ft. Zumwalt R-II',
 'Fort Osage R-I',
 'Francis Howell R-III',
 'Blue Springs R-Iv',
 'Excelsior Springs 40',
 'Sedalia 200',
 'Springfield R-Xii',
 'Columbia 93',
 'Mehlville R-Ix',
 'Caruthersville 18',
 'Kirkwood R-Vii',
 'Pattonville R-III',
 'Wentzville R-Iv',
 'Grain Valley R-V',
 'Park Hill',
 "Lee's Summit R-Vii",
 "Lee's Summit R-Vii",
 'Liberty 53',
 'Kirksville R-III',
 'Liberty 53',
 'Parkway C-2',
 'North Kansas City 74',
 'St. Joseph',
 'Kearney R-I',
 'Wentzville R-Iv',
 'Platte Co. R-III',
 '

In [44]:
# Check the length of all my list to make sure they match
print(len(page_links_list), len(district_list), len(city_list), len(clean_MO_ranking_final), 
      len(school_name_list), len(full_address_list_clean), len(phone_number_list), len(zipcode_list),
      len(clean_national_ranking_final), len(city_list_clean), len(state_list))

291 291 291 291 291 291 291 291 291 291 291


In [45]:
# Create the full schoold DF
schools_districts_df = pd.DataFrame({"school_name" : school_name_list,
                                     "school_distrct" : district_list,
                                     "MO_raniking" : clean_MO_ranking_final,
                                     "national_ranking" : clean_national_ranking_final,
                                     "city" : city_list_clean,
                                     "state" : state_list,
                                     "zipcode" : zipcode_list,
                                     "full_address" : full_address_list_clean,
                                     "phone_number" : phone_number_list,
                                     "school_page_link_usn" : page_links_list
                                    })


# Check DF
schools_districts_df

,school_name,school_distrct,MO_raniking,national_ranking,city,state,zipcode,full_address,phone_number,school_page_link_usn
0,Lafayette Sr. High,Rockwood R-Vi,1,431,Wildwood,MO,63011,"17050 Clayton Rd, Wildwood, Missouri 63011",(636) 733-4100,https://www.usnews.com//education/best-high-sc...
1,Smithville High,Smithville R-II,2,531,Smithville,MO,64089,"645 S Commercial Ave, Smithville, Missouri 64089",(816) 532-0405,https://www.usnews.com//education/best-high-sc...
2,Rockwood Summit Sr. High,Rockwood R-Vi,3,621,Fenton,MO,63026,"1780 Hawkins Rd, Fenton, Missouri 63026",(636) 891-6800,https://www.usnews.com//education/best-high-sc...
3,Eureka Sr. High,Rockwood R-Vi,4,681,Eureka,MO,63025,"4525 Hwy 109, Eureka, Missouri 63025",(636) 733-3100,https://www.usnews.com//education/best-high-sc...
4,Metro Academic and Classical High School,St. Louis Public Schools,5,799,St Louis,MO,63108,"4015 Mcpherson Ave, St Louis, Missouri 63108",(314) 534-3894,https://www.usnews.com//education/best-high-sc...
5,West High,Parkway C-2,6,823,Ballwin,MO,63011,"14653 Clayton Rd, Ballwin, Missouri 63011",(314) 415-7500,https://www.usnews.com//education/best-high-sc...
6,Lindbergh Sr. High,Lindbergh Schools,7,940,St. Louis,MO,63126,"5000 S. Lindbergh, St. Louis, Missouri 63126",(314) 729-2410,https://www.usnews.com//education/best-high-sc...
7,Marquette Sr. High,Rockwood R-Vi,8,947,Chesterfield,MO,63017,"2351 Clarkson Rd, Chesterfield, Missouri 63017",(636) 891-6000,https://www.usnews.com//education/best-high-sc...
8,Affton High,Affton 101,9,977,St Louis,MO,63123,"8309 Mackenzie Rd, St Louis, Missouri 63123",(314) 638-6330,https://www.usnews.com//education/best-high-sc...
9,Ft. Zumwalt West High,Ft. Zumwalt R-II,10,"1,196",O'fallon,MO,63366,"1251 Turtle Creek Dr, O'fallon, Missouri 63366",(636) 379-0300,https://www.usnews.com//education/best-high-sc...


In [112]:
# Export completed DF to CSV file 
export_schools_csv = schools_districts_df.to_csv (r'schools_districts.csv', header=True)

In [113]:
# Import ad read our csv outpts into DFs for futher use
houses_csv = "Scraped_houses.csv"
schools_csv = "schools_districts.csv"


houses_df = pd.read_csv(houses_csv)
schools_df = pd.read_csv(schools_csv)

In [114]:
# Check DF
houses_df.head()

,Unnamed: 0,Property Types,Price,Street Addresses,City,State,Zipcode,Sqft Area,Beds,Baths,Links
0,0,House for Sale,"$279,900","147 Country Creek Ct,",Ballwin,MO,63011,"2,001",3,2.5,https://www.realtor.com//realestateandhomes-de...
1,1,House for Sale,"$350,000","349 Cherry Hill Dr,",Ellisville,MO,63011,"2,397",3,2.5,https://www.realtor.com//realestateandhomes-de...
2,2,House for Sale,"$340,000","402 Baberton Dr,",Ballwin,MO,63011,"2,419",4,2.5,https://www.realtor.com//realestateandhomes-de...
3,3,House for Sale,"$319,900","302 Wildbrier Dr,",Ballwin,MO,63011,"2,319",5,2,https://www.realtor.com//realestateandhomes-de...
4,4,House for Sale,"$199,900","607 Morewood Ct,",Manchester,MO,63011,"1,376",3,2,https://www.realtor.com//realestateandhomes-de...


In [115]:
# Delet the unneeded column
del houses_df['Unnamed: 0']

In [116]:
# Strip the dollar signs from the housing prices for cleaner data
houses_df['Price'] = houses_df['Price'].str.strip('$')

In [117]:
# Re-Check DF
houses_df.head()

,Property Types,Price,Street Addresses,City,State,Zipcode,Sqft Area,Beds,Baths,Links
0,House for Sale,"279,900","147 Country Creek Ct,",Ballwin,MO,63011,"2,001",3,2.5,https://www.realtor.com//realestateandhomes-de...
1,House for Sale,"350,000","349 Cherry Hill Dr,",Ellisville,MO,63011,"2,397",3,2.5,https://www.realtor.com//realestateandhomes-de...
2,House for Sale,"340,000","402 Baberton Dr,",Ballwin,MO,63011,"2,419",4,2.5,https://www.realtor.com//realestateandhomes-de...
3,House for Sale,"319,900","302 Wildbrier Dr,",Ballwin,MO,63011,"2,319",5,2,https://www.realtor.com//realestateandhomes-de...
4,House for Sale,"199,900","607 Morewood Ct,",Manchester,MO,63011,"1,376",3,2,https://www.realtor.com//realestateandhomes-de...


In [131]:
# Export our cleaned houses DF to a CSV
export_houses_csv = schools_districts_df.to_csv (r'scraped_houses_cleaned.csv', header=True)

# 3. LOAD

## Now that our data is cleaned and ready for analysis, we want to make sure we store if for future use.  Due to the nature of our data retrieval through web scrapping, we decided to simply store the results into a Mongo DB for future use.  We summized that this would give us the most flexibility in storage and be an easy vehicle to allow us to easily store more scrapped data as we continue to collect.

In [132]:
# Import dependencies for the load
import pymongo
from pprint import pprint
import json

In [133]:
# Initialize PyMongo to work with MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [134]:
# Check if database exists already and drop it if so
dblist = client.list_database_names()
if "houses_schoools" in dblist:
    client.drop_database("houses_schoools")

In [135]:
# Create out DB and Collections
db = client.houses_schoools
house_collection = db.houses
schools_collection = db.schools_districts

In [136]:
# Create a list of dictionaries that hold the Mongo DB documents to be inserted
houses = houses_df.to_dict(orient='records')
schools = schools_df.to_dict(orient='records')


print(f"Posting {len(houses)} house records into collection of schools inside the 'houses_schoools' mongo database...")

print(f"Posting {len(schools)} school/ district records into collection of bestsellers inside the 'houses_schoools' mongo database...")


# Insert our houses and schools/ districts records indie teh MonDB DB for future analysis
house_collection.insert_many(houses)
schools_collection.insert_many(schools)

Posting 1239 house records into collection of schools inside the 'houses_schoools' mongo database...
Posting 291 school/ district records into collection of bestsellers inside the 'houses_schoools' mongo database...


In [137]:
# Check to make sure the house data was stored correctly
houses = house_collection.find()

for house in houses:
    print(house)

{'_id': ObjectId('5d29a44ae2d18c2732d367bb'), 'Property Types': 'House for Sale', 'Price': '279,900', 'Street Addresses': '147 Country Creek Ct,', 'City': 'Ballwin', 'State': 'MO', 'Zipcode': 63011, 'Sqft Area': '2,001', 'Beds': '3', 'Baths': '2.5', 'Links': 'https://www.realtor.com//realestateandhomes-detail/147-Country-Creek-Ct_Ballwin_MO_63011_M84542-50248'}
{'_id': ObjectId('5d29a44ae2d18c2732d367bc'), 'Property Types': 'House for Sale', 'Price': '350,000', 'Street Addresses': '349 Cherry Hill Dr,', 'City': 'Ellisville', 'State': 'MO', 'Zipcode': 63011, 'Sqft Area': '2,397', 'Beds': '3', 'Baths': '2.5', 'Links': 'https://www.realtor.com//realestateandhomes-detail/349-Cherry-Hill-Dr_Ellisville_MO_63011_M73739-32097'}
{'_id': ObjectId('5d29a44ae2d18c2732d367bd'), 'Property Types': 'House for Sale', 'Price': '340,000', 'Street Addresses': '402 Baberton Dr,', 'City': 'Ballwin', 'State': 'MO', 'Zipcode': 63011, 'Sqft Area': '2,419', 'Beds': '4', 'Baths': '2.5', 'Links': 'https://www.rea

In [138]:
# Check to make sure the school data was stored correctly
schools = schools_collection.find()

for school in schools:
    print(school)

{'_id': ObjectId('5d29a44ae2d18c2732d36c92'), 'Unnamed: 0': 0, 'school_name': 'Lafayette Sr. High', 'school_distrct': 'Rockwood R-Vi', 'MO_raniking': '1', 'national_ranking': '431', 'city': 'Wildwood', 'state': 'MO', 'zipcode': 63011, 'full_address': '17050 Clayton Rd, Wildwood, Missouri 63011', 'phone_number': '(636) 733-4100', 'school_page_link_usn': 'https://www.usnews.com//education/best-high-schools/missouri/districts/rockwood-r-vi/lafayette-sr-high-11883'}
{'_id': ObjectId('5d29a44ae2d18c2732d36c93'), 'Unnamed: 0': 1, 'school_name': 'Smithville High', 'school_distrct': 'Smithville R-II', 'MO_raniking': '2', 'national_ranking': '531', 'city': 'Smithville', 'state': 'MO', 'zipcode': 64089, 'full_address': '645 S Commercial Ave, Smithville, Missouri 64089', 'phone_number': '(816) 532-0405', 'school_page_link_usn': 'https://www.usnews.com//education/best-high-schools/missouri/districts/smithville-r-ii/smithville-high-11910'}
{'_id': ObjectId('5d29a44ae2d18c2732d36c94'), 'Unnamed: 0':